In [432]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
print(folium.__version__)

0.14.0


In [433]:
data = pd.read_excel('2022 지역별 전력사용량.xlsx')
data

,시군구별 계약종별별 전력사용량,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(단위 : kWh)
1,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
2,2022,서울특별시,종로구,주택용,24799032,24030840,21082506,19922174,19122306,20446064,26658320,30565429,24289712,20038789,19815027,22161770
3,2022,서울특별시,종로구,일반용,117205665,110074405,95738047,87398244,87152665,97931230,115665110,121707499,104182406,88013895,87213918,108162883
4,2022,서울특별시,종로구,교육용,11064107,10246892,8434552,7525722,6594830,7803407,9486678,9540822,8797864,7191532,7124513,9731798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2022,세종특별자치시,세종시,합 계,-298712777,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784
1833,2022,황해북도,개성시,주택용,0,0,0,0,0,0,0,0,0,0,0,0
1834,2022,황해북도,개성시,일반용,0,0,0,0,0,0,0,0,0,0,0,0
1835,2022,황해북도,개성시,산업용,0,0,0,0,0,0,0,0,0,0,0,0


In [434]:
data = data.iloc[1:, :]

In [435]:
data1 = data.rename(columns=data.iloc[0])
data1

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
1,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
2,2022,서울특별시,종로구,주택용,24799032,24030840,21082506,19922174,19122306,20446064,26658320,30565429,24289712,20038789,19815027,22161770
3,2022,서울특별시,종로구,일반용,117205665,110074405,95738047,87398244,87152665,97931230,115665110,121707499,104182406,88013895,87213918,108162883
4,2022,서울특별시,종로구,교육용,11064107,10246892,8434552,7525722,6594830,7803407,9486678,9540822,8797864,7191532,7124513,9731798
5,2022,서울특별시,종로구,산업용,6881320,6721003,6586374,6277347,5650865,6102450,6834738,6724399,6270983,5633842,5493366,5979578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1832,2022,세종특별자치시,세종시,합 계,-298712777,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784
1833,2022,황해북도,개성시,주택용,0,0,0,0,0,0,0,0,0,0,0,0
1834,2022,황해북도,개성시,일반용,0,0,0,0,0,0,0,0,0,0,0,0
1835,2022,황해북도,개성시,산업용,0,0,0,0,0,0,0,0,0,0,0,0


In [436]:
data1 = data1.drop(data1.index[0]).reset_index()
data1

,index,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2,2022,서울특별시,종로구,주택용,24799032,24030840,21082506,19922174,19122306,20446064,26658320,30565429,24289712,20038789,19815027,22161770
1,3,2022,서울특별시,종로구,일반용,117205665,110074405,95738047,87398244,87152665,97931230,115665110,121707499,104182406,88013895,87213918,108162883
2,4,2022,서울특별시,종로구,교육용,11064107,10246892,8434552,7525722,6594830,7803407,9486678,9540822,8797864,7191532,7124513,9731798
3,5,2022,서울특별시,종로구,산업용,6881320,6721003,6586374,6277347,5650865,6102450,6834738,6724399,6270983,5633842,5493366,5979578
4,6,2022,서울특별시,종로구,농사용,4955,4422,3828,3607,1876,1211,1301,987,772,716,960,625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,1832,2022,세종특별자치시,세종시,합 계,-298712777,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784
1831,1833,2022,황해북도,개성시,주택용,0,0,0,0,0,0,0,0,0,0,0,0
1832,1834,2022,황해북도,개성시,일반용,0,0,0,0,0,0,0,0,0,0,0,0
1833,1835,2022,황해북도,개성시,산업용,0,0,0,0,0,0,0,0,0,0,0,0


In [437]:
data1 = data1.drop(['index'], axis='columns')
data1

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2022,서울특별시,종로구,주택용,24799032,24030840,21082506,19922174,19122306,20446064,26658320,30565429,24289712,20038789,19815027,22161770
1,2022,서울특별시,종로구,일반용,117205665,110074405,95738047,87398244,87152665,97931230,115665110,121707499,104182406,88013895,87213918,108162883
2,2022,서울특별시,종로구,교육용,11064107,10246892,8434552,7525722,6594830,7803407,9486678,9540822,8797864,7191532,7124513,9731798
3,2022,서울특별시,종로구,산업용,6881320,6721003,6586374,6277347,5650865,6102450,6834738,6724399,6270983,5633842,5493366,5979578
4,2022,서울특별시,종로구,농사용,4955,4422,3828,3607,1876,1211,1301,987,772,716,960,625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,2022,세종특별자치시,세종시,합 계,-298712777,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784
1831,2022,황해북도,개성시,주택용,0,0,0,0,0,0,0,0,0,0,0,0
1832,2022,황해북도,개성시,일반용,0,0,0,0,0,0,0,0,0,0,0,0
1833,2022,황해북도,개성시,산업용,0,0,0,0,0,0,0,0,0,0,0,0


In [438]:
data1['1월'][1830] = 341596071
data1['1월'][1824] = 540525

In [439]:
data1

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2022,서울특별시,종로구,주택용,24799032,24030840,21082506,19922174,19122306,20446064,26658320,30565429,24289712,20038789,19815027,22161770
1,2022,서울특별시,종로구,일반용,117205665,110074405,95738047,87398244,87152665,97931230,115665110,121707499,104182406,88013895,87213918,108162883
2,2022,서울특별시,종로구,교육용,11064107,10246892,8434552,7525722,6594830,7803407,9486678,9540822,8797864,7191532,7124513,9731798
3,2022,서울특별시,종로구,산업용,6881320,6721003,6586374,6277347,5650865,6102450,6834738,6724399,6270983,5633842,5493366,5979578
4,2022,서울특별시,종로구,농사용,4955,4422,3828,3607,1876,1211,1301,987,772,716,960,625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,2022,세종특별자치시,세종시,합 계,341596071,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784
1831,2022,황해북도,개성시,주택용,0,0,0,0,0,0,0,0,0,0,0,0
1832,2022,황해북도,개성시,일반용,0,0,0,0,0,0,0,0,0,0,0,0
1833,2022,황해북도,개성시,산업용,0,0,0,0,0,0,0,0,0,0,0,0


In [440]:
data2 = data1[data1['계약종별'] == '합 계']
data2

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
7,2022,서울특별시,종로구,합 계,163322953,154459935,134597421,123998502,122507867,137709225,166310833,175846293,148813409,123453812,121687271,148532525
14,2022,서울특별시,중구,합 계,213129833,198087817,176903377,162626263,168159612,188922717,228535962,237629451,201147307,166894770,162823855,200338633
22,2022,서울특별시,용산구,합 계,137560380,132110989,114570962,107874423,107725453,119537976,146398102,160821270,136286659,113598637,109860621,128644854
30,2022,서울특별시,성동구,합 계,206805972,197703858,176603681,168066382,159556376,174827870,206122492,225764973,197419781,168767617,165644186,187766380
38,2022,서울특별시,동대문구,합 계,137692775,134281938,116272608,108725374,100707951,109798543,134400341,151913031,130222573,107300299,104867374,121505998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806,2022,경상남도,합천군,합 계,45636602,44733986,38465802,34305782,29169802,32617619,34618202,36619276,33292960,30032086,36205270,37485997
1814,2022,제주특별자치도,제주시,합 계,317382760,312863560,282631093,279015706,260799390,275897191,333992302,374992798,325234704,274007485,271833514,291704756
1822,2022,제주특별자치도,서귀포시,합 계,235970974,247996771,219438140,216066207,191378889,182401818,197566647,221712922,197541741,170996896,169411175,194517951
1830,2022,세종특별자치시,세종시,합 계,341596071,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784


In [441]:
data2 = data2.reset_index().drop(['index'], axis='columns')

In [442]:
data2

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,2022,서울특별시,종로구,합 계,163322953,154459935,134597421,123998502,122507867,137709225,166310833,175846293,148813409,123453812,121687271,148532525
1,2022,서울특별시,중구,합 계,213129833,198087817,176903377,162626263,168159612,188922717,228535962,237629451,201147307,166894770,162823855,200338633
2,2022,서울특별시,용산구,합 계,137560380,132110989,114570962,107874423,107725453,119537976,146398102,160821270,136286659,113598637,109860621,128644854
3,2022,서울특별시,성동구,합 계,206805972,197703858,176603681,168066382,159556376,174827870,206122492,225764973,197419781,168767617,165644186,187766380
4,2022,서울특별시,동대문구,합 계,137692775,134281938,116272608,108725374,100707951,109798543,134400341,151913031,130222573,107300299,104867374,121505998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2022,경상남도,합천군,합 계,45636602,44733986,38465802,34305782,29169802,32617619,34618202,36619276,33292960,30032086,36205270,37485997
226,2022,제주특별자치도,제주시,합 계,317382760,312863560,282631093,279015706,260799390,275897191,333992302,374992798,325234704,274007485,271833514,291704756
227,2022,제주특별자치도,서귀포시,합 계,235970974,247996771,219438140,216066207,191378889,182401818,197566647,221712922,197541741,170996896,169411175,194517951
228,2022,세종특별자치시,세종시,합 계,341596071,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784


In [443]:
data2['합계'] = data2.iloc[:, 4:].sum(axis=1)
pd.options.display.float_format = '{:.5f}'.format

In [444]:
data3 = data2.copy()
data3

,연도,시도,시군구,계약종별,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,합계
0,2022,서울특별시,종로구,합 계,163322953,154459935,134597421,123998502,122507867,137709225,166310833,175846293,148813409,123453812,121687271,148532525,1721240046.00000
1,2022,서울특별시,중구,합 계,213129833,198087817,176903377,162626263,168159612,188922717,228535962,237629451,201147307,166894770,162823855,200338633,2305199597.00000
2,2022,서울특별시,용산구,합 계,137560380,132110989,114570962,107874423,107725453,119537976,146398102,160821270,136286659,113598637,109860621,128644854,1514990326.00000
3,2022,서울특별시,성동구,합 계,206805972,197703858,176603681,168066382,159556376,174827870,206122492,225764973,197419781,168767617,165644186,187766380,2235049568.00000
4,2022,서울특별시,동대문구,합 계,137692775,134281938,116272608,108725374,100707951,109798543,134400341,151913031,130222573,107300299,104867374,121505998,1457688805.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2022,경상남도,합천군,합 계,45636602,44733986,38465802,34305782,29169802,32617619,34618202,36619276,33292960,30032086,36205270,37485997,433183384.00000
226,2022,제주특별자치도,제주시,합 계,317382760,312863560,282631093,279015706,260799390,275897191,333992302,374992798,325234704,274007485,271833514,291704756,3600355259.00000
227,2022,제주특별자치도,서귀포시,합 계,235970974,247996771,219438140,216066207,191378889,182401818,197566647,221712922,197541741,170996896,169411175,194517951,2445000131.00000
228,2022,세종특별자치시,세종시,합 계,341596071,320317372,313436948,303255300,290989455,301368688,333617467,351081461,329434500,298635306,307210784,331444784,3822388136.00000


In [445]:
data3[['시군구', '합계']]

,시군구,합계
0,종로구,1721240046.00000
1,중구,2305199597.00000
2,용산구,1514990326.00000
3,성동구,2235049568.00000
4,동대문구,1457688805.00000
...,...,...
225,합천군,433183384.00000
226,제주시,3600355259.00000
227,서귀포시,2445000131.00000
228,세종시,3822388136.00000


In [446]:
data4 = data3[['시도', '시군구', '합계']]

In [447]:
data4

,시도,시군구,합계
0,서울특별시,종로구,1721240046.00000
1,서울특별시,중구,2305199597.00000
2,서울특별시,용산구,1514990326.00000
3,서울특별시,성동구,2235049568.00000
4,서울특별시,동대문구,1457688805.00000
...,...,...,...
225,경상남도,합천군,433183384.00000
226,제주특별자치도,제주시,3600355259.00000
227,제주특별자치도,서귀포시,2445000131.00000
228,세종특별자치시,세종시,3822388136.00000


In [448]:
data4.to_excel('시군구_합계.xlsx') #편집으로 코드 다 붙여줌 나 정말 미쳐버리겠다~

In [449]:
data5 = pd.read_excel('시군구_합계_최종.xlsx')


In [450]:
data5 = data5.iloc[:, 1:]

In [451]:
data5

,SUM,CODE
0,4822165527,11680
1,1659002911,11740
2,933057384,11305
3,2752726888,11500
4,1735923906,11620
...,...,...
224,300610905,48870
225,433183384,48890
226,3600355259,50110
227,2445000131,50130


In [452]:
state_geo = 'sig_WGS84.json'

In [453]:
import json
json_data=open(state_geo, encoding='utf-8').read()
jsonResult = json.loads(json_data)

In [454]:
jsonResult

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.012, 37.582],
      [127.008, 37.581],
      [127.007, 37.582],
      [127.007, 37.584],
      [127.007, 37.585],
      [127.007, 37.586],
      [127.004, 37.588],
      [127.003, 37.588],
      [127.003, 37.589],
      [127.002, 37.589],
      [127.002, 37.59],
      [127.001, 37.59],
      [127.001, 37.591],
      [127.002, 37.591],
      [127.001, 37.592],
      [127, 37.592],
      [126.999, 37.592],
      [126.998, 37.592],
      [126.997, 37.592],
      [126.996, 37.592],
      [126.995, 37.592],
      [126.994, 37.592],
      [126.989, 37.592],
      [126.988, 37.592],
      [126.987, 37.593],
      [126.986, 37.593],
      [126.986, 37.594],
      [126.985, 37.594],
      [126.984, 37.594],
      [126.983, 37.595],
      [126.982, 37.595],
      [126.981, 37.595],
      [126.981, 37.596],
      [126.979, 37.596],
      [126.979, 37.597],
      [126.978, 3

In [455]:
# 중앙위치 계산
def center_calc(points_df):
    x = points_df.x
    y = points_df.y

    X = (max(x)+min(x))/2.
    Y = (max(y)+min(y))/2.

    return X, Y

In [456]:
# 다중 Array 구조 이중으로 변환
def points_array(points):

    final_points = []

    for x in range(0, len(points)):

        if len(points[x]) == 2:
            final_points.append(points[x])
        else:
            target = points[x]
            for y in range(0, len(target)):
                final_points.append(target[y])

    return final_points

In [457]:
center_locations = pd.DataFrame()
codes = []
names = []
x_list = []
y_list = []
for x in range(0, len(jsonResult['features'])):
    code = jsonResult['features'][x]['properties']['SIG_CD']
    name = jsonResult['features'][x]['properties']['SIG_KOR_NM']
    # 중앙값 생성
    points = jsonResult['features'][x]['geometry']['coordinates'][0]
    points = points_array(points)
    points_df = pd.DataFrame(points)
    points_df.columns = ['x','y']
    X, Y = center_calc(points_df)

    # 결과
    codes.append(code)
    names.append(name)
    x_list.append(X)
    y_list.append(Y)

# 데이터 프레임 생성
center_locations['CODE'] = codes
center_locations['NAME'] = names
center_locations['X'] = x_list
center_locations['Y'] = y_list

In [458]:
center_locations

,CODE,NAME,X,Y
0,11110,종로구,126.98600,37.59900
1,11140,중구,126.99450,37.55800
2,11170,용산구,126.98150,37.53150
3,11200,성동구,127.04150,37.55050
4,11215,광진구,127.08550,37.54850
...,...,...,...,...
245,51790,화천군,127.67250,38.14100
246,51800,양구군,128.01200,38.16550
247,51810,인제군,128.24700,38.09600
248,51820,고성군,128.42500,38.38900


In [459]:
data5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   SUM     229 non-null    int64
 1   CODE    229 non-null    int64
dtypes: int64(2)
memory usage: 3.7 KB


In [460]:
center_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CODE    250 non-null    object 
 1   NAME    250 non-null    object 
 2   X       250 non-null    float64
 3   Y       250 non-null    float64
dtypes: float64(2), object(2)
memory usage: 7.9+ KB


In [461]:
data5.CODE = data5.CODE.astype(str) 
#state_data.CODE.dtype

In [462]:
data5 = data5.rename(columns={'시군구':'NAME', '합계':'SUM'})

In [463]:
target_df = pd.merge(data5,center_locations, how='left',on='CODE')
target_df

,SUM,CODE,NAME,X,Y
0,4822165527,11680,강남구,127.06650,37.49600
1,1659002911,11740,강동구,127.14600,37.54850
2,933057384,11305,강북구,127.01500,37.64700
3,2752726888,11500,강서구,126.82350,37.56550
4,1735923906,11620,관악구,126.94400,37.46600
...,...,...,...,...,...
224,300610905,48870,함양군,127.73150,35.54350
225,433183384,48890,합천군,128.16700,35.60000
226,3600355259,50110,제주시,126.22700,33.40850
227,2445000131,50130,서귀포시,126.26750,33.11800


In [464]:
import numpy as np
target_df = target_df[~np.isnan(target_df['X'])] # 정보 없는 값 제외

In [465]:
target_df

,SUM,CODE,NAME,X,Y
0,4822165527,11680,강남구,127.06650,37.49600
1,1659002911,11740,강동구,127.14600,37.54850
2,933057384,11305,강북구,127.01500,37.64700
3,2752726888,11500,강서구,126.82350,37.56550
4,1735923906,11620,관악구,126.94400,37.46600
...,...,...,...,...,...
224,300610905,48870,함양군,127.73150,35.54350
225,433183384,48890,합천군,128.16700,35.60000
226,3600355259,50110,제주시,126.22700,33.40850
227,2445000131,50130,서귀포시,126.26750,33.11800


In [466]:
# Initialize the map:
m = folium.Map(location=[36, 127], zoom_start=7)


# I can add marker one by one on the map
for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    folium.Circle(location, radius=target_df.iloc[i]['SUM']/600000,color='red',fill_color='red', popup=target_df.iloc[i]['NAME']).add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

m

In [467]:
# Save to html
#m.save(os.path.join('kr_energy.html'))
#m
# webbrowser.open_new("kr_energy.html")

In [468]:
state_data = pd.read_csv('Total_People_2023_2.csv', encoding = 'euc-kr')
state_data

,CODE,POPULATION
0,0,51377213
1,11,9409466
2,11110,140032
3,11140,121482
4,11170,216547
...,...,...
287,48880,60184
288,48890,41531
289,50,676489
290,50110,492379


In [469]:
center_locations.CODE.dtype == state_data.CODE.dtype

False

In [470]:
state_data.CODE = state_data.CODE.astype(str) 
#state_data.CODE.dtype

In [471]:
center_locations.CODE.dtype == state_data.CODE.dtype

True

In [472]:
target_df = pd.merge(state_data,center_locations,on='CODE')
target_df

,CODE,POPULATION,NAME,X,Y
0,11110,140032,종로구,126.98600,37.59900
1,11140,121482,중구,126.99450,37.55800
2,11170,216547,용산구,126.98150,37.53150
3,11200,278750,성동구,127.04150,37.55050
4,11215,336446,광진구,127.08550,37.54850
...,...,...,...,...,...
245,48870,37117,함양군,127.73150,35.54350
246,48880,60184,거창군,127.88700,35.71200
247,48890,41531,합천군,128.16700,35.60000
248,50110,492379,제주시,126.22700,33.40850


In [473]:
target_df = target_df[~np.isnan(target_df['X'])] # 위치 정보 없는 값 제외

In [474]:
target_df

,CODE,POPULATION,NAME,X,Y
0,11110,140032,종로구,126.98600,37.59900
1,11140,121482,중구,126.99450,37.55800
2,11170,216547,용산구,126.98150,37.53150
3,11200,278750,성동구,127.04150,37.55050
4,11215,336446,광진구,127.08550,37.54850
...,...,...,...,...,...
245,48870,37117,함양군,127.73150,35.54350
246,48880,60184,거창군,127.88700,35.71200
247,48890,41531,합천군,128.16700,35.60000
248,50110,492379,제주시,126.22700,33.40850


In [475]:
# Initialize the map:

# I can add marker one by one on the map
for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    folium.CircleMarker(location, radius=target_df.iloc[i]['POPULATION']/25000,color='green',fill_color='green', popup=target_df.iloc[i]['NAME']).add_to(m)


folium.LayerControl(collapsed=False).add_to(m)



In [476]:
# Save to html
m.save(os.path.join('kr_energy_population1.html'))
m
# webbrowser.open_new("kr_energy.html")